## Modal demo

This notebook uses Modal to run code remotely. Before running this notebook, you need to authenticate:

```bash
uv run modal setup
```

Then restart the notebook kernel.

### Local metrics

Let's get some metrics displayed right here in the notebook! We'll define a function to draw a loss chart. This function will be called several times during training, and it should update the chart each time.

In [ ]:
from dataclasses import dataclass
# from typing import cast


@dataclass
class Metrics:
    epoch: int
    loss: float


def plot_history(history: list[Metrics]):
    import matplotlib.pyplot as plt

    xs = [h.epoch for h in history]
    ys = [h.loss for h in history]

    fig, ax = plt.subplots(figsize=(8, 4))
    ax.set_title('Training progress')
    ax.set_xlabel('Epoch')
    ax.set_ybound(0, 1)
    ax.plot(xs, ys, label='Loss')
    ax.legend()
    plt.close(fig)
    return fig

### Remote function

Here we define a mock training function that will run remotely. It just loops a few times and returns a stub model function.

We specify the exact packages that we'll need in the image to keep it small. Version specifiers are needed (see [`freeze`](src/dair/requirements.py)), so that:
- The remote function behaves exactly how it would locally
- Objects can be pickled and sent back and forth.

We'll add `modal` itself as a dependency, because it's used by `emit_metrics` (see _Training_ below).


In [ ]:
from time import sleep
import modal

from mini.experiment import Experiment
from utils.requirements import freeze


exp = Experiment('demo')
exp.image = modal.Image.debian_slim().pip_install(freeze('modal')).add_local_python_source('utils')

history: list[Metrics] = []


@exp.run_hither
def track(metrics: Metrics, display_id: str):
    from utils.nb import show

    history.append(metrics)
    fig = plot_history(history)
    show(fig, handle=display_id)


@exp.run_thither(gpu=None)
async def train(epochs: int):
    from uuid import uuid4 as uuid

    print('Training...')

    display_id = str(uuid())
    for i in range(epochs):
        track(Metrics(epoch=i + 1, loss=1 / (i + 1)), display_id)
        sleep(0.2)

    def stub_model(x):
        if x == 'What is your quest?':
            return 'To seek the Holy Grail.'
        elif x == 'What is the air-speed velocity of an unladen swallow?':
            return 'What do you mean? An African or European swallow?'
        else:
            return "I don't know that!"

    print('Training complete')
    return stub_model

Prevent build logs from showing in the next step. Currently Modal doesn't provide a way to separate the build logs from container stdout.

In [ ]:
@exp.run_thither
async def prebuild():
    pass


async with exp.run():
    await prebuild()

### Training

Now let's run the training code remotely.

A [distributed `Queue`](https://modal.com/docs/reference/modal.Queue) is used to send progress information back during training. You can push rich data onto the queue (like actual Matplotlib figures), and it transparently handles serialization - but in this example, a simple dataclass is emitted. The progress function is wrapped in [`send_to`](src/dair/comms.py): a context manager that provides a simple interface over the queue.

If we only cared about the final result, or if we were happy just printing progress to stdout, we could call `train` synchronously.

In [ ]:
async with exp.run():
    model = await train(20)

### Testing

The model was created remotely, serialized, and sent back. Now we can run it locally!

In [ ]:
from textwrap import dedent

x = 'What is your quest?'
print(
    dedent(f"""
    {x}
    {model(x)}
    """).strip()
)

x = 'What is the air-speed velocity of an unladen swallow?'
print(
    dedent(f"""
    {x}
    {model(x)}
    {model(model(x))}
    """).strip()
)